# Train Sentiment Analysis

Here we'll train a sentiment analysis model to validate the data from the API.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path
import pandas as pd

In [5]:
sentiment140_path = Path('../datasets/sentiment140/sentiment140.csv')
data = pd.read_csv(sentiment140_path, encoding = "ISO-8859-1")

In [6]:
data.head()

,sentiment,id,date,query_string,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## Data preprocessing

Preprocess the texts:
- Convert to Lowercase: Convert all characters from the text to lowercase
- Remove special characters: Remove links and usernames and transform emojis to text
- Remove repetitions: Remove char repetitions (e.g. whaaaaaat => what)
- Remove Stop words: Remove common stop words

In [7]:
import re
from time import time
import nltk
from emoji import demojize

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leena\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
texts = data.tweet

start = time()
# Lowercasing
texts = texts.str.lower()

# Remove special chars
texts = texts.str.replace(r"(http|@)\S+", "")
texts = texts.apply(demojize)
texts = texts.str.replace(r"::", ": :")
texts = texts.str.replace(r"’", "'")
texts = texts.str.replace(r"[^a-z\':_]", " ")

# Remove repetitions
pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
texts = texts.str.replace(pattern, r"\1")

# Transform short negation form
texts = texts.str.replace(r"(can't|cannot)", 'can not')
texts = texts.str.replace(r"n't", ' not')

# Remove stop words
stopwords = nltk.corpus.stopwords.words('english')
stopwords.remove('not')
stopwords.remove('nor')
stopwords.remove('no')
texts = texts.apply(
    lambda x: ' '.join([word for word in x.split() if word not in stopwords])
)

print("Time to clean up: {:.2f} sec".format(time() - start))

data.tweet = texts

Time to clean up: 564.98 sec


## Tokenize

Transform the text corpus to a vector representation

- **num_words**: Number of words to use

In [9]:
num_words = 10000

In [10]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer

In [11]:
tokenizer = Tokenizer(num_words=num_words, lower=True)
tokenizer.fit_on_texts(data.tweet)

file_to_save = Path('../datasets/sentiment140/tokenizer.pickle').resolve()
with file_to_save.open('wb') as file:
    pickle.dump(tokenizer, file)

## Split data

Split the dataset in train and validation data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train = pd.DataFrame(columns=['sentiment', 'tweet'])
validation = pd.DataFrame(columns=['sentiment', 'tweet'])
for label in data.sentiment.unique():
    label_data = data[data.sentiment == label]
    train_data, validation_data = train_test_split(label_data, test_size=0.3)
    train = pd.concat([train, train_data])
    validation = pd.concat([validation, validation_data])

## Model

Define the Bidirectional GRU model

In [14]:
from tensorflow.keras.layers import Input, Embedding, GRU
from tensorflow.keras.layers import Dropout, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Dense
from tensorflow.keras.models import Sequential

In [15]:
input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
embedding_dim = 200
input_length = 100
gru_units = 128
gru_dropout = 0.1
recurrent_dropout = 0.1
dropout = 0.1

In [16]:
model = Sequential()
model.add(Embedding(
    input_dim=input_dim,
    output_dim=embedding_dim,
    input_shape=(input_length,)
))

model.add(Bidirectional(GRU(
    gru_units,
    return_sequences=True,
    dropout=gru_dropout,
    recurrent_dropout=recurrent_dropout
)))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(dropout))

model.add(Dense(1, activation='sigmoid'))

In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 200)          2000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 256)          253440    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                8224      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 2,261,697
Trainable params: 2,261,697
Non-trainable params: 0
______________________________________________

## Prepare the data

Prepare the model input data

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
train_sequences = [text.split() for text in train.tweet]
validation_sequences = [text.split() for text in validation.tweet]
list_tokenized_train = tokenizer.texts_to_sequences(train_sequences)
list_tokenized_validation = tokenizer.texts_to_sequences(validation_sequences)

x_train = pad_sequences(list_tokenized_train, maxlen=input_length)
x_validation = pad_sequences(list_tokenized_validation, maxlen=input_length)
y_train = train.sentiment.replace(4, 1)
y_validation = validation.sentiment.replace(4, 1)

## Train model

Do the training process with the given data

In [20]:
batch_size = 128
epochs = 1

In [21]:
model.fit(
    x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_validation, y_validation),
)

8750/8750 [==============================] - 14757s 2s/step - loss: 0.4355 - accuracy: 0.7976 - val_loss: 0.4153 - val_accuracy: 0.8092


In [22]:
model_file = Path('../models/sentiment_analysis/gru_model.h5').resolve()
model.save_weights(model_file.as_posix())